In [ ]:
# A5 小書建立器（直接可用版：files/ 下載 + BytesIO "wb" + ZIP 檢查）
# 需求：python-docx, pillow, ipywidgets

import io, re, hashlib, zipfile
from pathlib import Path
from docx import Document
from docx.shared import Mm, Pt
from docx.enum.section import WD_SECTION
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

try:
    from PIL import Image
    _PIL_OK = True
except Exception:
    _PIL_OK = False

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# --- 下載：以 files/<name>.docx 提供；不使用 FileLink，不暴露 /home/jovyan/ ---
def save_and_link(doc, filename="a5_book.docx"):
    buf = io.BytesIO()
    doc.save(buf)          # 先完整寫到記憶體（docx 是 zip）
    buf.seek(0)
    outpath = Path.cwd() / filename
    with open(outpath, "wb") as f:   # ✅ 二進位
        f.write(buf.read())

    size = outpath.stat().st_size
    sha  = hashlib.sha256(outpath.read_bytes()).hexdigest()[:12]

    # ZIP 結構自檢（避免壞檔）
    ok = False
    detail = ""
    try:
        with zipfile.ZipFile(outpath, "r") as z:
            names = set(z.namelist())
            needed = {"[Content_Types].xml", "_rels/.rels", "word/document.xml"}
            missing = [n for n in needed if n not in names]
            ok = not missing
            if not ok:
                detail = "缺少：" + ", ".join(missing)
            else:
                detail = "結構完整"
    except Exception as e:
        ok = False
        detail = f"不是合法 zip：{e}"

    if ok:
        display(HTML(
            f"<div style='border:1px solid #0a0;padding:12px;border-radius:8px;margin:8px 0'>"
            f"<b>完成！</b> 已輸出：<code>{outpath.name}</code>（{size} bytes，SHA-256：<code>{sha}…</code>，{detail}）"
            f"</div>"
            f"<a href='files/{outpath.name}' download='{outpath.name}' "
            f"style='background:#4CAF50;color:white;padding:8px 14px;border-radius:6px;text-decoration:none;display:inline-block;'>下載 {outpath.name}</a>"
        ))
    else:
        display(HTML(
            f"<div style='border:1px solid #c33;padding:12px;border-radius:8px;margin:8px 0;color:#c00'>"
            f"⚠️ 檔案異常：{detail}</div>"
            f"<a href='files/{outpath.name}' download='{outpath.name}' "
            f"style='background:#999;color:white;padding:8px 14px;border-radius:6px;text-decoration:none;display:inline-block;'>下載（檢體）</a>"
        ))

# --- 版面 ---
A5_W, A5_H = Mm(148), Mm(210)
MARGIN = Mm(15)
IMG_MAX_W_MM_INNER = 118
IMG_MAX_H_MM_INNER = 120
IMG_MAX_W_MM_COVER = 118
IMG_MAX_H_MM_COVER = 140
IMG_MAX_W_MM_BACK  = 118
IMG_MAX_H_MM_BACK  = 140

def set_section_to_a5(section):
    section.page_width = A5_W
    section.page_height = A5_H
    section.left_margin = MARGIN
    section.right_margin = MARGIN
    section.top_margin = MARGIN
    section.bottom_margin = MARGIN

def add_page_number_footer(section, prefix="第 ", suffix=" 頁", start_at_1=True):
    section.footer.is_linked_to_previous = False
    if start_at_1:
        sectPr = section._sectPr
        pgNumType = sectPr.find(qn('w:pgNumType'))
        if pgNumType is None:
            pgNumType = OxmlElement('w:pgNumType'); sectPr.append(pgNumType)
        pgNumType.set(qn('w:start'), "1")
    ft = section.footer
    for p in list(ft.paragraphs): p._element.getparent().remove(p._element)
    p = ft.add_paragraph(); p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    if prefix: p.add_run(prefix)
    fldChar1 = OxmlElement('w:fldChar'); fldChar1.set(qn('w:fldCharType'), 'begin')
    instrText = OxmlElement('w:instrText'); instrText.set(qn('xml:space'),'preserve'); instrText.text=' PAGE '
    fldChar2 = OxmlElement('w:fldChar'); fldChar2.set(qn('w:fldCharType'), 'separate')
    fldText = OxmlElement('w:t'); fldText.text = "1"
    fldChar3 = OxmlElement('w:fldChar'); fldChar3.set(qn('w:fldCharType'), 'end')
    r = p.add_run()
    for el in (fldChar1,instrText,fldChar2,fldText,fldChar3): r._r.append(el)
    if suffix: p.add_run(suffix)

def _extract_name_and_bytes(record):
    if not isinstance(record, dict): return (None, None)
    meta = record.get('metadata') or {}
    name = meta.get('name') or record.get('name')
    data = record.get('content') if record.get('content') is not None else record.get('data')
    return (name, data)

def fileupload_get_list(uploader):
    v = uploader.value; items = []
    if not v: return items
    if isinstance(v,(tuple,list)):
        for rec in v:
            name, blob = _extract_name_and_bytes(rec)
            if name and blob is not None: items.append((name, blob))
    elif isinstance(v, dict):
        for _, rec in v.items():
            name, blob = _extract_name_and_bytes(rec)
            if name and blob is not None: items.append((name, blob))
    return items

def compute_scaled_width_mm(image_bytes, max_w_mm, max_h_mm, scale=1.0):
    max_w_mm *= scale; max_h_mm *= scale; width_mm_to_use = max_w_mm
    if _PIL_OK:
        try:
            from PIL import Image
            with Image.open(io.BytesIO(image_bytes)) as im:
                w,h = im.size
                if w>0 and h>0:
                    r = w/h; width_by_height_cap = max_h_mm * r
                    width_mm_to_use = min(max_w_mm, width_by_height_cap)
        except Exception:
            width_mm_to_use = max_w_mm
    return width_mm_to_use

def add_picture_to_paragraph(paragraph, image_bytes, width_mm, align_center=True):
    run = paragraph.add_run(); run.add_picture(io.BytesIO(image_bytes), width=Mm(width_mm))
    if align_center: paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

def add_single_image_block(doc, image_bytes, max_w_mm, max_h_mm, scale=1.0):
    width_mm = compute_scaled_width_mm(image_bytes, max_w_mm, max_h_mm, scale=scale)
    p = doc.add_paragraph(); add_picture_to_paragraph(p, image_bytes, width_mm, True)

def add_two_images_row(doc, left_bytes, right_bytes, each_max_w_mm, max_h_mm, left_scale=1.0, right_scale=1.0):
    tbl = doc.add_table(rows=1, cols=2); tbl.autofit = True
    if left_bytes:
        wl = compute_scaled_width_mm(left_bytes, each_max_w_mm, max_h_mm, scale=left_scale)
        add_picture_to_paragraph(tbl.rows[0].cells[0].paragraphs[0], left_bytes, wl, True)
    if right_bytes:
        wr = compute_scaled_width_mm(right_bytes, each_max_w_mm, max_h_mm, scale=right_scale)
        add_picture_to_paragraph(tbl.rows[0].cells[1].paragraphs[0], right_bytes, wr, True)

_marker_pattern = re.compile(r'(\[(?:圖片|左圖|右圖)(?:\d+)?%?\])')
def parse_marker(token):
    m = re.match(r'^\[(圖片|左圖|右圖)(\d+)?%?\]$', token)
    if not m: return (None,None)
    kind = m.group(1); perc = float(m.group(2))/100.0 if m.group(2) else 1.0
    return {"圖片":"single","左圖":"left","右圖":"right"}.get(kind), perc

def render_marked_block(doc, raw_text, images, start_idx, font_size_pt, max_w_mm, max_h_mm, base_scale=1.0):
    idx = start_idx; used_marker = False; buffer_left = None
    tokens = _marker_pattern.split(str(raw_text))
    for tk in tokens:
        if not tk: continue
        kind, local = parse_marker(tk)
        if kind == "single":
            used_marker = True
            if idx < len(images):
                _, ibytes = images[idx]; add_single_image_block(doc, ibytes, max_w_mm, max_h_mm, base_scale*local); idx += 1
        elif kind in ("left","right"):
            used_marker = True
            if kind == "left":
                if idx < len(images): buffer_left = (images[idx][1], base_scale*local); idx += 1
                else: buffer_left = (None, base_scale*local)
            else:  # right
                left_bytes, left_scale = buffer_left if buffer_left else (None, base_scale)
                right_bytes = images[idx][1] if idx < len(images) else None
                if idx < len(images): idx += 1
                each_max_w = (max_w_mm - 6) / 2.0
                add_two_images_row(doc, left_bytes, right_bytes, each_max_w, max_h_mm, left_scale, base_scale*local)
                buffer_left = None
        else:
            text_block = tk.strip()
            if text_block:
                for para in text_block.split("\n\n"):
                    p = doc.add_paragraph(para.strip())
                    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
                    p.paragraph_format.space_after = Pt(6)
                    for r in p.runs: r.font.size = Pt(font_size_pt)
    if buffer_left and buffer_left[0] is not None:
        add_single_image_block(doc, buffer_left[0], max_w_mm, max_h_mm, buffer_left[1]); used_marker=True
    if (not used_marker) and (idx < len(images)):
        _, ibytes = images[idx]; add_single_image_block(doc, ibytes, max_w_mm, max_h_mm, base_scale); idx += 1
    return idx

def render_page(doc, raw_text, images, start_idx, font_size_pt, max_w_mm, max_h_mm, base_scale=1.0, pagebreak_after=True):
    idx = start_idx
    if bool(str(raw_text).strip()) or idx < len(images):
        idx = render_marked_block(doc, raw_text, images, idx, font_size_pt, max_w_mm, max_h_mm, base_scale)
        if pagebreak_after: doc.add_page_break()
        return idx
    return idx

# === UI ===
title = widgets.HTML("<h3>A5 小書 Word 產生器（直接可用版）</h3>")
cover_text = widgets.Textarea(value="我的小書\n副標題\n\n[圖片]\n也可用 [左圖80%][右圖50%]", description="封面文字",
                              layout=widgets.Layout(width="100%", height="140px"))
cover_imgs = widgets.FileUpload(accept="image/*", multiple=True, description="封面圖片（可多檔）")
inner_texts = widgets.Textarea(value="第一章開場……\n\n第二章轉折……\n\n[左圖80%][右圖60%]",
                               description="內頁文字", layout=widgets.Layout(width="100%", height="220px"))
inner_imgs = widgets.FileUpload(accept="image/*", multiple=True, description="內頁圖片（依序使用，不會被消耗）")
back_text = widgets.Textarea(value="封底簡介……\n\n[圖片60%]", description="封底文字",
                             layout=widgets.Layout(width="100%", height="140px"))
back_imgs = widgets.FileUpload(accept="image/*", multiple=True, description="封底圖片（可多檔）")

font_size = widgets.IntSlider(value=12, min=9, max=24, step=1, description="字級（全域）")
img_scale = widgets.FloatSlider(value=1.0, min=0.6, max=2.0, step=0.05, description="圖片縮放（全域）")
outfile_name = widgets.Text(value="a5_book.docx", description="輸出檔名")
add_numbers = widgets.Checkbox(value=True, description="內頁加頁碼（封面/封底不加）")

build_btn = widgets.Button(description="生成 (.docx)", button_style="success", icon="file")
log = widgets.Output()

ui = widgets.VBox([
    title,
    widgets.HTML("<b>封面</b>"),
    cover_text, cover_imgs,
    widgets.HTML("<hr>"),
    widgets.HTML("<b>內頁</b>"),
    inner_texts, inner_imgs,
    widgets.HTML("<hr>"),
    widgets.HTML("<b>封底</b>"),
    back_text, back_imgs,
    widgets.HTML("<hr>"),
    widgets.HBox([font_size, img_scale, add_numbers]),
    outfile_name,
    build_btn,
    log
])

def build_doc(_):
    with log:
        clear_output()
        safe_name = re.sub(r'[^A-Za-z0-9._-]+', '_', outfile_name.value or 'a5_book.docx')
        if not safe_name.lower().endswith(".docx"): safe_name += ".docx"
        outname = safe_name

        doc = Document()

        # 封面
        set_section_to_a5(doc.sections[0])
        cov_idx = render_page(doc, cover_text.value, fileupload_get_list(cover_imgs), 0,
                              font_size.value+2, IMG_MAX_W_MM_COVER, IMG_MAX_H_MM_COVER,
                              float(img_scale.value), pagebreak_after=False)

        # 內頁（加頁碼）
        inner_section = doc.add_section(WD_SECTION.NEW_PAGE)
        set_section_to_a5(inner_section)
        if add_numbers.value:
            add_page_number_footer(inner_section, "第 ", " 頁", True)

        idx = 0
        pages = [t.strip() for t in inner_texts.value.split("\n\n") if t.strip()] or [""]
        imgs = fileupload_get_list(inner_imgs)
        for tx in pages:
            idx = render_page(doc, tx, imgs, idx, font_size.value,
                              IMG_MAX_W_MM_INNER, IMG_MAX_H_MM_INNER, float(img_scale.value), True)

        # 封底（不加頁碼）
        back_section = doc.add_section(WD_SECTION.NEW_PAGE)
        set_section_to_a5(back_section)
        back_section.footer.is_linked_to_previous = False
        for p in list(back_section.footer.paragraphs): p._element.getparent().remove(p._element)
        _ = render_page(doc, back_text.value, fileupload_get_list(back_imgs), 0,
                        font_size.value, IMG_MAX_W_MM_BACK, IMG_MAX_H_MM_BACK, float(img_scale.value), False)

        save_and_link(doc, outname)

build_btn.on_click(build_doc)
display(ui)